<a href="https://colab.research.google.com/github/nnikolovskiii/Deep-learning-lab-exercises/blob/master/Citation_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color="lightgreen">Citation network dataset (benchmark dataset)</font>

Graph: The ogbn-arxiv dataset is a directed graph, representing the citation network between all Computer Science (CS) arXiv papers indexed by MAG [1]. Each node is an arXiv paper and each directed edge indicates that one paper cites another one. Each paper comes with a 128-dimensional feature vector obtained by averaging the embeddings of words in its title and abstract. The embeddings of individual words are computed by running the skip-gram model [2] over the MAG corpus. We also provide the mapping from MAG paper IDs into the raw texts of titles and abstracts here. In addition, all papers are also associated with the year that the corresponding paper was published.

Link: https://ogb.stanford.edu/docs/nodeprop/#ogbn-arxiv

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.1 MB/s eta 0:00:00


In [ ]:
!pip install ogb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=fa4e3de4d9aa684633702f2cd44ebfc3e3268db566713a2a34e009bf1155d080
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


Running everything on GPU

In [15]:
if torch.cuda.is_available():
        print("GPU is available.")
else:
        print("No GPU found. Running on CPU.")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

GPU is available.
Device: cuda


#<font color="lightgreen">GCN Model</font>

In the image it is shown the layers that are a part od this Graph convolutional neural network. The output layer is a soft classification, and the lost function is negative log likelihood, where it takes into account all of the predicted classes, and compares it to the target label.

The model is trained in 100 epochs. It is trained on the whole data, but it is trained, tested and evaluated, on a certain set of data. In the end the best model is saved, which has best score in validation phase.

The model has learned embeddings for the nodes in our netork and the next task will be to use the for classificaion!

![test](https://drive.google.com/uc?id=128AuYAXNXGg7PIhJJ7e420DoPWKb-RtL)

In [19]:
import torch
from torch_geometric.data import DataLoader

print(torch.__version__)
import copy

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import pandas as pd


class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GCN, self).__init__()

        self.convs = nn.ModuleList([GCNConv(input_dim, hidden_dim)] +
                                   [GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers - 2)]+
                                   [GCNConv(hidden_dim, output_dim)])

        self.bns = nn.ModuleList([nn.BatchNorm1d(hidden_dim) for _ in range(num_layers - 1)])
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

        # The log softmax layer

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < len(self.bns):
                x = self.bns[i](x)
                x = torch.relu(x)
                x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)

        if self.return_embeds:
            return x
        else:
            x = F.log_softmax(x, dim=1)
            return x


def train(model, data, train_idx, optimizer, loss_fn):
    model.train()
    optimizer.zero_grad()

    out = model(data.x.to(device), data.edge_index.to(device))

    # Slice the model's output and target labels using train_idx
    train_output = out[train_idx]
    train_target = data.y[train_idx]

    # Calculate the loss
    loss = loss_fn(train_output, train_target.flatten())

    # Backpropagation
    loss.backward()

    # Update the model's weights
    optimizer.step()

    # Return the loss as a float
    return loss.item()


@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    # Set the model to evaluation mode
    model.eval()

    # Forward pass: compute the model's predictions on the entire dataset
    out = model(data.x.to(device), data.edge_index.to(device))

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
        print("Saving Model Predictions")

        data = {'y_pred': y_pred.view(-1).cpu().detach().numpy()}

        df = pd.DataFrame(data=data)
        # Save locally as csv
        df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)

    return train_acc, valid_acc, test_acc


if __name__ == '__main__':


    dataset_name = 'ogbn-arxiv'
    data = PygNodePropPredDataset(name=dataset_name)
    data.x = data.x.to(device)
    data.y = data.y.to(device)

    split_idx = data.get_idx_split()
    train_loader = DataLoader(split_idx["train"], batch_size=32, shuffle=True)
    valid_loader = DataLoader(split_idx["valid"], batch_size=32, shuffle=False)
    test_loader = DataLoader(split_idx["test"], batch_size=32, shuffle=False)

    train_idx = split_idx['train'].to(device)
    tmp = data.num_classes


    args = {
        'num_layers': 3,
        'hidden_dim': 256,
        'dropout': 0.5,
        'lr': 0.01,
        'epochs': 100,
        'device' : device
    }


    model = GCN(data.num_features, args['hidden_dim'],
                data.num_classes, args['num_layers'],
                args['dropout'])
    model = model.to(device)
    evaluator = Evaluator(name='ogbn-arxiv')

    # reset the parameters to initial random value
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
    loss_fn = F.nll_loss

    best_model = None
    best_valid_acc = 0
    print(data)
    for epoch in range(1, 1 + args["epochs"]):
        loss = train(model, data, train_idx, optimizer, loss_fn)
        result = test(model, data, split_idx, evaluator)  # Evaluate on the validation set
        train_acc, valid_acc, test_acc = result
        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            best_model = copy.deepcopy(model)
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

    best_result = test(best_model, data, split_idx, evaluator, save_model_results=True)
    train_acc, valid_acc, test_acc = best_result
    print(f'Best model: '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')

2.1.0+cu118
PygNodePropPredDataset()


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 01, Loss: 4.0895, Train: 6.05%, Valid: 4.58% Test: 3.86%
Epoch: 02, Loss: 2.9380, Train: 15.05%, Valid: 15.23% Test: 14.09%
Epoch: 03, Loss: 2.5421, Train: 20.30%, Valid: 25.13% Test: 24.05%
Epoch: 04, Loss: 2.3887, Train: 30.44%, Valid: 36.20% Test: 33.10%
Epoch: 05, Loss: 2.2684, Train: 37.07%, Valid: 42.43% Test: 40.27%
Epoch: 06, Loss: 2.1728, Train: 38.66%, Valid: 44.01% Test: 42.51%
Epoch: 07, Loss: 2.1057, Train: 38.43%, Valid: 43.51% Test: 43.28%
Epoch: 08, Loss: 2.0463, Train: 38.68%, Valid: 43.71% Test: 43.83%
Epoch: 09, Loss: 1.9987, Train: 38.84%, Valid: 44.06% Test: 44.57%
Epoch: 10, Loss: 1.9484, Train: 39.39%, Valid: 44.64% Test: 45.14%
Epoch: 11, Loss: 1.9131, Train: 40.44%, Valid: 45.18% Test: 45.69%
Epoch: 12, Loss: 1.8858, Train: 42.10%, Valid: 45.84% Test: 46.28%
Epoch: 13, Loss: 1.8618, Train: 44.66%, Valid: 47.64% Test: 47.01%
Epoch: 14, Loss: 1.8333, Train: 47.82%, Valid: 49.82% Test: 47.67%
Epoch: 15, Loss: 1.8058, Train: 50.49%, Valid: 51.43% Test: 48.00